<a href="https://colab.research.google.com/github/Tank86092/2025ML/blob/main/2025ML_HW1_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2025 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Optimizer: RMSProp (default)
* TODOs:
  - Complete the `valid()` function
  - Tune the hyperparameters in `train_config`
  - Implement 2nd-order polynomial regression model (without interaction terms) in `minibatch_2()`
  - Implement feature normalization in `normalize_train_data()`
  - Feature selection



# **Import Some Packages**

In [33]:
import numpy as np
import csv
import math
import pandas as pd
import os

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!**


In [34]:
import time
seed = time.time().as_integer_ratio()[0] % (2**32 - 1)
seed = 335144792
print("Random Seed:", seed)
np.random.seed(seed)
# feats = [14]
# 335144792 loss = 2.93708
# 289140301 loss = 0.22291424418973896
# 469040481 loss =  2.90900
# 2483168760 loss =  0.21035390971386417
# feats = [11,14]
# 335144792 loss = 2.83189
# 2186126077
# 2810199398
# 3417990017


Random Seed: 335144792


# **Download training data**


In [35]:
# !gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"
# !gdown --id '155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
#!gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

In [36]:
def valid(x, y,norm_params):
  if y > 20:
    return False
  return True


# Create your dataset
def parse2train(data, feats,norm_params=None):

  x = []
  y = []

  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - 8

  for i in range(total_length):
    x_tmp = data[feats,i:i+8 ] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
    y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5
    # Filter out extreme values to train.
    if valid(x_tmp, y_tmp,norm_params):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)

  x = np.array(x)
  y = np.array(y)

  return x,y



#**Gradient descent**
###**RMSProp**
1. $v_t=\beta \cdot v_{t-1} + (1-\beta)(\nabla w_t)^2$
2. $w_{t+1}=w_t - \frac{\eta}{\sqrt{(v_t)}+\epsilon}\nabla w_t$




* This is our gradient descent algorithm. RMSProp was implemented in `minibatch()`.
* You can implement other algorithm, such as SGD or other gradient descent variants listed below, which may (or may not) improve performance.
* However, **modules like sklearn and pytorch are not allowed!!!**
* Ref:
  - Prof. G. Hinton's lecture: https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
  - Prof. Hung-Yi Lee's video: https://youtu.be/HYUXEeh3kwY?si=RtLjSj51WK1pmz87

###**Adam (RMSProp + Momemtum)**
* Ref:
  - Paper: https://arxiv.org/pdf/1412.6980
  - Prof. Hung-Yi Lee's video: https://youtu.be/HYUXEeh3kwY?si=RtLjSj51WK1pmz87

###**AdamW (Adam with decoupled weight decay)**
* Ref:
  - Paper: https://arxiv.org/pdf/1711.05101




In [37]:
def minibatch(x, y, config):
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    print(x.shape, y.shape)

    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    epoch = config.epoch
    decay_rate = config.decay_rate
    epsilon = 1e-8

    # Linear regression: only contains two parameters (w, b).
    # w = np.full(x.shape[1], 0.1).reshape(-1, 1)
    w = np.random.uniform(-1,1,x[0].shape).reshape(-1, 1)
    bias = 0.1

    # Optimizer states
    cache_w = np.zeros_like(w)
    cache_b = 0.0

    # Training loop
    for num in range(epoch):
        loss_sum = 0
        for b in range(int(x.shape[0] / batch_size)):
            x_batch = x[b * batch_size:(b + 1) * batch_size]
            y_batch = y[b * batch_size:(b + 1) * batch_size].reshape(-1, 1)

            # Prediction of linear regression
            pred = np.dot(x_batch, w) + bias

            # Loss
            loss = y_batch - pred
            loss_sum += (loss**2).sum()

            # Compute gradient
            g_t = np.dot(x_batch.transpose(), loss) * (-2)
            g_t_b = loss.sum(axis=0) * (-2)

            # Update cache
            cache_w = decay_rate * cache_w + (1 - decay_rate) * g_t**2
            cache_b = decay_rate * cache_b + (1 - decay_rate) * g_t_b**2

            # Update weight & bias
            w -= lr * g_t / (np.sqrt(cache_w) + epsilon)
            bias -= lr * g_t_b / (np.sqrt(cache_b) + epsilon)
        print("Epoch %d/%d" % (num+1, epoch),end=' ')
        print("loss = ", np.sqrt(loss_sum/x.shape[0]))

    return w, bias

# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch_2(x, y, config):
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    x2 = x ** 2
    x = np.concatenate((x, x2), axis=1)
    y = y[index]

    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    epoch = config.epoch
    decay_rate = config.decay_rate
    epsilon = 1e-8

    # Linear regression: only contains two parameters (w, b).
    # w = np.full(x.shape[1], 0.1).reshape(-1, 1)
    w = np.random.uniform(-1,1,x[0].shape).reshape(-1, 1)
    bias = 0

    # Optimizer states
    cache_w = np.zeros_like(w)
    cache_b = 0.0

    # Training loop
    for num in range(epoch):
        loss_sum = 0
        for b in range(int(x.shape[0] / batch_size)):
            x_batch = x[b * batch_size:(b + 1) * batch_size]
            y_batch = y[b * batch_size:(b + 1) * batch_size].reshape(-1, 1)

            # Prediction of 2nd order regression
            pred = np.dot(x_batch, w) + bias

            # Loss
            loss = y_batch - pred
            loss_sum += (loss**2).sum()

            # Compute gradient
            g_t = np.dot(x_batch.transpose(), loss) * (-2)
            g_t_b = loss.sum(axis=0) * (-2)

            # Update cache
            cache_w = decay_rate * cache_w + (1 - decay_rate) * g_t**2
            cache_b = decay_rate * cache_b + (1 - decay_rate) * g_t_b**2

            # Update weight & bias
            w -= lr * g_t / (np.sqrt(cache_w) + epsilon)
            bias -= lr * g_t_b / (np.sqrt(cache_b) + epsilon)
        print("Epoch %d/%d" % (num+1, epoch),end=' ')
        print("loss = ", np.sqrt(loss_sum/x.shape[0]))

    return w, bias

In [38]:
from argparse import Namespace

# TODO: Tune the config to boost your performance.
train_config = Namespace(
    batch_size = 16,
    lr = 0.003,
    epoch = 30,
    decay_rate = 0.90
)

# **Training your regression model**

In [39]:
train_df = pd.read_csv("./content/train.csv")
train_df

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,10.8,0.32,1.7,8.6,10.3,22.9,21,0.6,0.0,71,1.9,172,171,0.6,15
1,10.8,0.27,1.6,6.2,7.8,23.8,20,1.4,0.0,71,1.7,161,129,1.8,13
2,11.0,0.25,0.9,5.4,6.3,27.4,21,0.8,0.0,68,1.6,152,147,1.5,12
3,11.0,0.23,0.7,3.1,3.8,29.5,21,1.8,0.0,68,1.6,138,145,1.7,9
4,11.3,0.22,0.8,2.9,3.8,30.7,16,1.9,0.0,67,1.6,140,139,1.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,29.0,0.41,1.2,14.0,15.3,23.0,21,1.4,0.0,74,2.8,149,168,2.0,14
5770,28.2,0.33,1.7,11.7,13.5,19.5,23,2.1,0.0,78,2.3,187,179,2.5,15
5771,28.0,0.29,1.3,9.1,10.4,17.6,17,1.5,0.0,78,2.0,173,200,1.5,13
5772,28.0,0.27,1.4,9.5,11.0,15.4,17,1.1,0.0,75,1.8,171,135,0.9,10


In [40]:
# TODO: Normalize each column (except PM2.5) for the report (use z-score normalization)
def normalize_train_data(df):

    mean = df.mean(axis=0)
    std = df.std(axis=0)

    data_norm = df.copy()
    for col in df.columns:
        if col != 'PM2.5':
            data_norm[col] = (df[col] - mean[col]) / std[col]
            
    norm_params = {}
    mean = mean.to_dict()
    std = std.to_dict()
    for key in mean.keys():
        if key != 'PM2.5':
            norm_params[key] = {'mean': mean[key], 'std': std[key]}
            
    return data_norm, norm_params

# caclulate the correlation of each feature to PM2.5
correlations = train_df.corr()['PM2.5'].abs().sort_values(ascending=False)
print("Feature correlations to PM2.5:\n", correlations)
# calculate the skewness of features
skewness = train_df.skew().sort_values(ascending=False)
print("Feature skewness:\n", skewness)

Feature correlations to PM2.5:
 PM2.5         1.000000
PM10          0.818868
CO            0.659148
NO2           0.554274
NOx           0.513650
SO2           0.361333
O3            0.233924
NO            0.227219
AMB_TEMP      0.176147
WD_HR         0.171932
WIND_DIREC    0.137658
WS_HR         0.102047
WIND_SPEED    0.101197
RH            0.081576
RAINFALL      0.060801
Name: PM2.5, dtype: float64
Feature skewness:
 RAINFALL      16.229383
SO2           16.009668
NO             6.989056
PM10           3.347849
NOx            2.615972
NO2            1.995609
PM2.5          1.878909
CO             1.641582
WS_HR          1.048011
WIND_SPEED     0.953726
O3             0.818826
WD_HR          0.564390
WIND_DIREC     0.516856
AMB_TEMP      -0.402460
RH            -0.533830
dtype: float64


In [41]:
# Choose your features to train.
# Hint:
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.
# 0:AMB_TEMP,1:CO, 2:NO,3:NO2,4:NOx,5:O3,6:PM10,7:WS_HR,8:RAINFALL,9:RH,10:SO2,11:WD_HR,12:WIND_DIREC,13:WIND_SPEED,14:PM2.5

# feats = [11,14]
feats = [2]

In [42]:
# Training data preprocessing
def train_processing(train_df, norm=False):
    """Process training train_df with optional normalization"""

    if norm:
        # Normalize training data and save parameters (mean & std)
        data_norm, norm_params = normalize_train_data(train_df)
        data_values = data_norm.values
    else:
        # Use raw training data
        data_values = train_df.values
        norm_params = None

    # Common processing steps
    train_data = np.transpose(np.array(np.float64(data_values)))
    train_x, train_y = parse2train(train_data, feats,norm_params)

    return train_x, train_y, norm_params

train_x, train_y, norm_params = train_processing(train_df, norm=True)
# optimal linear regression parameters using least square 
X = np.concatenate((train_x, np.ones((train_x.shape[0], 1))), axis=1)
w_optimal = np.linalg.pinv(X) @ train_y
loss = np.sqrt(np.mean((train_y - (X @ w_optimal))**2))
print("Optimal least square loss:", loss)

# optimal 2nd order regression parameters using least square 
X = np.concatenate((train_x ** 2,train_x, np.ones((train_x.shape[0], 1))), axis=1)
w_optimal = np.linalg.pinv(X) @ train_y
loss = np.sqrt(np.mean((train_y - (X @ w_optimal))**2))
print("Optimal 2nd order least square loss:", loss)
# caclulate the correlation of each input to answer
for i in range(train_x.shape[1]):
    corr = np.corrcoef(train_x[:, i], train_y)[0, 1]
    print(f"Feature {i} correlation with PM2.5: {corr:.4f}")



Optimal least square loss: 5.032653242054937
Optimal 2nd order least square loss: 5.005800476301345
Feature 0 correlation with PM2.5: 0.0588
Feature 1 correlation with PM2.5: 0.0647
Feature 2 correlation with PM2.5: 0.0781
Feature 3 correlation with PM2.5: 0.0901
Feature 4 correlation with PM2.5: 0.0848
Feature 5 correlation with PM2.5: 0.0839
Feature 6 correlation with PM2.5: 0.0769
Feature 7 correlation with PM2.5: 0.0814


In [43]:
# Train your regression model
w, bias = minibatch_2(train_x, train_y, train_config)
# display parameters
print("Weight:", w.reshape(-1,))
print("Bias:", bias)

Epoch 1/30 loss =  13.037449580632622
Epoch 2/30 loss =  12.235858278248205
Epoch 3/30 loss =  11.582198284207903
Epoch 4/30 loss =  11.001347747569266
Epoch 5/30 loss =  10.450256474447805
Epoch 6/30 loss =  9.91339762010956
Epoch 7/30 loss =  9.384417600812956
Epoch 8/30 loss =  8.860658095565332
Epoch 9/30 loss =  8.347970355315411
Epoch 10/30 loss =  7.852113067929145
Epoch 11/30 loss =  7.381117189230313
Epoch 12/30 loss =  6.928895388783283
Epoch 13/30 loss =  6.506640664981648
Epoch 14/30 loss =  6.125682282856525
Epoch 15/30 loss =  5.805679290223104
Epoch 16/30 loss =  5.554390217727851
Epoch 17/30 loss =  5.3677031522303915
Epoch 18/30 loss =  5.2303500274264705
Epoch 19/30 loss =  5.142052977002718
Epoch 20/30 loss =  5.095717865393999
Epoch 21/30 loss =  5.07406892203742
Epoch 22/30 loss =  5.063333915685417
Epoch 23/30 loss =  5.056984855083659
Epoch 24/30 loss =  5.0526010421335386
Epoch 25/30 loss =  5.04921073397095


Epoch 26/30 loss =  5.046470277216588
Epoch 27/30 loss =  5.044392595949647
Epoch 28/30 loss =  5.043124847983775
Epoch 29/30 loss =  5.042145303133397
Epoch 30/30 loss =  5.0411470491110135


# **Testing:**


In [44]:
def parse2test(data, feats):
  x = []
  y = []
  for i in range(90):
    x_tmp = data[feats,8*i: 8*i+8]
    x.append(x_tmp.reshape(-1,))
    if i == 89:
      # The last one is just a placeholder
      y_tmp = 0
    else:
      y_tmp = data[-1, 8*i+8] # last column of (i+8)th row: PM2.5
    y.append(y_tmp)


  # x.shape: (n, 15, 8)
  x = np.array(x)
  y = np.array(y)
  return x, y

In [45]:
def normalize_test_data(df, norm_params):
    data_norm = df.copy()

    for col, params in norm_params.items():
        if col in df.columns:
            data_norm[col] = (df[col] - params['mean']) / params['std']

    return data_norm

In [46]:
test_df = pd.read_csv('./content/test.csv')
test_df

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,27.5,0.22,0.7,9.0,9.8,13.2,31.0,1.2,0.0,79.0,1.7,180.0,171.0,1.2,20.0
1,27.2,0.17,0.4,5.0,5.4,15.7,20.0,1.5,0.0,79.0,1.6,192.0,187.0,1.9,8.0
2,26.8,0.17,0.4,4.3,4.8,12.8,16.0,1.6,0.0,81.0,1.3,181.0,180.0,1.8,9.0
3,26.7,0.19,0.4,4.1,4.5,12.0,21.0,1.7,0.0,80.0,1.5,179.0,188.0,2.3,6.0
4,26.4,0.22,0.4,4.1,4.6,10.1,23.0,2.2,0.0,81.0,1.5,184.0,186.0,1.9,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,16.0,0.26,0.3,3.9,4.2,47.1,34.0,2.7,0.0,70.0,0.5,130.0,133.0,2.8,13.0
716,15.6,0.25,0.4,3.3,3.7,44.1,27.0,3.0,0.0,74.0,0.6,136.0,131.0,3.2,15.0
717,15.7,0.24,0.4,3.7,4.1,44.1,29.0,2.9,0.0,73.0,0.6,133.0,129.0,2.7,12.0
718,15.1,0.24,0.6,10.5,11.1,29.9,9.0,0.8,0.0,95.0,0.6,24.0,21.0,1.1,8.0


In [47]:
# Testing data preprocessing
def test_processing(test_df, norm=False, norm_params=norm_params):
    if norm:
        if norm_params is None:
            raise ValueError("norm_params required when " \
            "rue")
        # Apply training normalization parameters to testing data
        data_norm = normalize_test_data(test_df, norm_params)
        data_values = data_norm.values
    else:
        # Use raw testing data
        data_values = test_df.values

    # Common processing steps
    test_data = np.transpose(np.array(np.float64(data_values)))
    test_x, test_y = parse2test(test_data, feats)

    return test_x, test_y 

test_x, test_y = test_processing(test_df, norm=True, norm_params=norm_params)

# **Write result as .csv**

---



In [48]:
test_opt_df = pd.read_csv('./my_sol_opt.csv')
test_y = test_opt_df['Predicted'].values

with open('my_sol.csv', 'w', newline='') as csvf:
  writer = csv.writer(csvf)
  writer.writerow(['Id','Predicted'])

  loss_sum = 0.0
  for i in range(int(test_x.shape[0])):
    # Prediction of linear regression
    if test_x.shape[1] != w.shape[0]:
        x2 = test_x[i] ** 2
        test_x_i = np.concatenate((test_x[i], x2), axis=0)
    else:
        test_x_i = test_x[i]
    prediction = (np.dot(np.reshape(w,-1),test_x_i) + bias)[0]
    # print(f"ans: {test_y[i]}, prediction: {prediction}")
    loss_sum += (test_y[i] - prediction) ** 2
    writer.writerow([i, prediction])
  print("Test loss = ", np.sqrt(loss_sum/test_x.shape[0]))

Test loss =  3.340084282926679
